In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os, random

from modules.painting import painting
from modules.database import database as db

from keras.models import Sequential, Model
from keras.layers.core import (Dropout, Flatten, Dense, Activation)
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import (RandomFlip, RandomRotation, Input, BatchNormalization, 
                          RandomTranslation, RandomZoom)
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.applications.vgg19 import VGG19, preprocess_input

from tensorflow import unique_with_counts

#### Control Variables

In [3]:
# setting path variables for Working Directory and folder to save and load
#   models from.
wd = os.getcwd()
WD_PATH =  os.path.abspath(wd)
PATH_TRAINING = os.path.join(WD_PATH, "model_training")

# # initializing the database object
gallery = db()

# ### How many pictures should be used per artist
TOTAL_PAINTINGS = 3971
SIZE_TRAINING = 190
SIZE_TESTING = 38
SIZE_UNUSED = 3971 - ((SIZE_TESTING + SIZE_TRAINING) * 10 )
PIXEL_SIZE = 128

#### Loading data from DB

In [43]:
# checking the available paintings per artist.
num_paintings = np.zeros(10,np.int16)
for j in range(10):
    num_paintings[j] = len(gallery.get_paintingids_from_artist(j+1))
print(num_paintings)

[877 439 336 291 259 239 702 262 255 311]


In [24]:
## creating arrays to hold the pictures taken from the DB
testing_images = np.zeros((10*SIZE_TESTING,PIXEL_SIZE,PIXEL_SIZE,3))
index_testing = 0
skip_count_testing = 0
training_images = np.zeros((10*SIZE_TRAINING,PIXEL_SIZE,PIXEL_SIZE,3))
index_training = 0
skip_count_training = 0
# creating the arrays to hold labels. In this case they are the artist ids.
# testing_labels = np.zeros((10*SIZE_TESTING,1),dtype=int)
# training_labels = np.zeros((10*SIZE_TRAINING,1),dtype=int)
testing_labels = np.array([0]*SIZE_TESTING*10,dtype=int)
training_labels = np.array([0]*SIZE_TRAINING*10,dtype=int)
unused_paintings = np.array([0]*SIZE_UNUSED,dtype=int)


# filling the arrays with picture arrays. They will be resized according
# to the pixel_size value
unused_index = 0
for i in range(1,11):
    ids = gallery.get_paintingids_from_artist(i)
    # random.seed(1983)
    random.shuffle(ids)
    print("New Artist")
    ids_training = ids[ : SIZE_TRAINING]
    # print(len(ids_training))
    ids_testing = ids[SIZE_TRAINING : SIZE_TRAINING + SIZE_TESTING]
    
    # print(len(ids_testing))
    ids_unused = ids[SIZE_TRAINING + SIZE_TESTING : ]
    # print(len(ids_unused))
    ##  Größen der arrays correct
    # print("start:", unused_index)
    # print("end :", unused_index+len(ids_unused))
    for l, f in zip(range(unused_index,unused_index+len(ids_unused))
                    , ids_unused):
        unused_paintings[l] = f[0]
    unused_index += len(ids_unused)
    # print("After adding the Artist", unused_index)
    # print(unused_paintings.shape)
    for j in ids_testing:
        temp_p = painting("local DB", id=j[0])
        temp_p_res = cv.resize(temp_p.ndarray, dsize=(PIXEL_SIZE,PIXEL_SIZE)
                               ,interpolation=cv.INTER_CUBIC)
        if temp_p_res.shape == (PIXEL_SIZE,PIXEL_SIZE,3):
            testing_images[index_testing] = temp_p_res
            testing_labels[index_testing] = temp_p.artist_id-1
            # testing_labels[index_testing] = [temp_p.artist_id-1,]
            index_testing += 1
        else:
            skip_count_testing += 1
    # print("Testing images status")
    # print(index_testing)
    # print(skip_count_testing)
    for k in ids_training:
        temp_p = painting("local DB", id=k[0])
        temp_p_res = cv.resize(temp_p.ndarray, dsize=(PIXEL_SIZE,PIXEL_SIZE)
                               ,interpolation=cv.INTER_CUBIC)
        if temp_p_res.shape == (PIXEL_SIZE,PIXEL_SIZE,3):
            training_images[index_training] = temp_p_res
            training_labels[index_training] = temp_p.artist_id-1
            # training_labels[index_training] = [temp_p.artist_id-1,]
            index_training += 1
        else:
            skip_count_training += 1
    print("Training images status")
    print(index_training)
    print(skip_count_training)

# ## dropping the last few array positions of testing images, which where 
# ## not filled.
testing_images = testing_images[:index_testing,:,:,:]
testing_labels = testing_labels[:index_testing]
# ## dropping the last few array positions of training images, which where 
# ## not filled.
training_images = training_images[:index_training,:,:,:]
training_labels = training_labels[:index_training]

# # the pixels on an image are rescaled from 0-255 to 0-1 
training_images, testing_images = training_images/255, testing_images/255 

# defining labels in a list
class_names = [i[1] for i in gallery.get_all_artists()]

New Artist
Training images status
190
0
New Artist
Training images status
372
8
New Artist
Training images status
562
8
New Artist
Training images status
683
77
New Artist
Training images status
872
78
New Artist
Training images status
1061
79
New Artist
Training images status
1251
79
New Artist
Training images status
1441
79
New Artist
Training images status
1630
80
New Artist
Training images status
1820
80


#### Checking data shapes

In [25]:
print(training_images.shape)
print(testing_images.shape)
print(training_labels.shape)
print(testing_labels.shape)
print(type(training_labels[185:200]))

(1820, 128, 128, 3)
(362, 128, 128, 3)
(1820,)
(362,)
<class 'numpy.ndarray'>


***
Creating the weight vector for the artists.

#### Checking data counts

In [26]:
y_tl, idx_tl, count_tl = unique_with_counts(training_labels)
y_testl, idx_testl, count_testl = unique_with_counts(testing_labels)
y_unused, idx_unused, count_unused = unique_with_counts(unused_paintings)


In [27]:
print(y_tl, count_tl)
print(y_testl, count_testl)
print(unique_with_counts(count_unused)[2])

tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32) tf.Tensor([190 182 190 121 189 189 190 190 189 190], shape=(10,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32) tf.Tensor([37 36 38 25 38 36 38 38 38 38], shape=(10,), dtype=int32)
tf.Tensor([1691], shape=(1,), dtype=int32)


***
## Modelling part
####  creating the model


In [33]:
base_model = VGG19(include_top = False,
                   classes = 10, 
                   input_shape = (PIXEL_SIZE, PIXEL_SIZE, 3))

base_model.trainable = False
base_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [34]:
# Layer for the model. 
## data augmentation counteracting the small number of paintings
data_augmentation = Sequential([
    RandomFlip('horizontal'),
    RandomFlip('vertical'),
    RandomRotation(0.2),
    RandomZoom(0.1),
    RandomTranslation(0.1, 0.1),
])
prediction = Sequential([
    Flatten(),
    Dense(512),
    BatchNormalization(),
    Dropout(0.1),
    Dense(512),
    Dense(10, activation = 'softmax'),
])

In [35]:
# Creating the model from basemodel and the other layers.
inputs = Input(shape=(PIXEL_SIZE, PIXEL_SIZE, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x)
outputs = prediction(x)
model = Model(inputs, outputs)

In [36]:
# compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 sequential_2 (Sequential)   (None, 128, 128, 3)       0         
                                                                 
 tf.__operators__.getitem_1   (None, 128, 128, 3)      0         
 (SlicingOpLambda)                                               
                                                                 
 tf.nn.bias_add_1 (TFOpLambd  (None, 128, 128, 3)      0         
 a)                                                              
                                                                 
 vgg19 (Functional)          (None, 4, 4, 512)         20024384  
                                                                 
 sequential_3 (Sequential)   (None, 10)                4464

In [38]:
## training the model
epochs = 100
batch_size = 16

early_stopping = EarlyStopping(patience = 20, 
                               verbose = 2, 
                               restore_best_weights = True)
    
history = model.fit(training_images,
                    training_labels,
                    validation_data = (testing_images, testing_labels),
                    # class_weight = class_weights,
                    epochs = epochs,
                    batch_size = batch_size,
                    callbacks = early_stopping)

Epoch 1/100
114/114 [==============================] - 76s 666ms/step - loss: 2.6014 - accuracy: 0.2505 - val_loss: 32.7918 - val_accuracy: 0.1050
Epoch 2/100
114/114 [==============================] - 76s 671ms/step - loss: 2.3821 - accuracy: 0.2681 - val_loss: 28.2884 - val_accuracy: 0.1050
Epoch 3/100
114/114 [==============================] - 77s 680ms/step - loss: 2.1361 - accuracy: 0.2945 - val_loss: 24.3743 - val_accuracy: 0.1243
Epoch 4/100
114/114 [==============================] - 76s 669ms/step - loss: 2.2124 - accuracy: 0.2824 - val_loss: 72.9142 - val_accuracy: 0.1050
Epoch 5/100
114/114 [==============================] - 79s 694ms/step - loss: 2.0901 - accuracy: 0.2896 - val_loss: 29.5103 - val_accuracy: 0.1050
Epoch 6/100
114/114 [==============================] - 78s 688ms/step - loss: 1.9886 - accuracy: 0.3016 - val_loss: 25.2036 - val_accuracy: 0.1050
Epoch 7/100
  4/114 [>.............................] - ETA: 1:01 - loss: 2.2965 - accuracy: 0.1875

KeyboardInterrupt: 

In [ ]:
model.save(os.path.join(PATH_TRAINING,"image_classifier_BvD_withVGG19basemodel.model"))

In [ ]:
model = models.load_model(os.path.join(PATH_TRAINING,"image_classifier_BvD_withVGG19basemodel.model"))

#### check one prediction

In [ ]:
test_painting = painting("local DB",random.choice(unused_paintings))
print(test_painting.id)
test_ndarray = cv.resize(test_painting.ndarray, dsize=(PIXEL_SIZE,PIXEL_SIZE)
                               ,interpolation=cv.INTER_CUBIC)
test_ndarray = test_ndarray/255
temp_array = np.zeros((1,PIXEL_SIZE,PIXEL_SIZE,3))
print(temp_array.shape)
temp_array[0] = test_ndarray
prediction = model.predict(temp_array)
np.set_printoptions(suppress=True)
print(np.round(prediction, 4))
print(np.argmax(prediction))
index = np.argmax(prediction)
print(f"Prediction is {class_names[index]}")
imgplot = plt.imshow(test_painting.ndarray)
plt.show()